In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from sporadic import *

In [10]:
import Task, TaskSet

In [65]:
ts = TaskSet.TaskSet(
    [
        Task.Task(O=0, T=25, D=25, X=0, C=(2, 2)),
        Task.Task(O=0, T=26, D=26, X=1, C=(4, 5)),
#        Task.Task(O=0, T=27, D=27, X=1, C=(5, 6)),
#        Task.Task(O=0, T=27, D=27, X=1, C=(5, 6)),
    ]
)

In [69]:
ss = get_initial_state(ts.get_df())
ss

,nat,rct,crit,O,T,D,X,0,1,U0,U1,t_id,ss_id,done,wl,sched_id,request
0,0.0,0,0,0.0,25.0,25.0,0.0,2.0,2.0,0.080000,0.080000,0,0,True,0.0,-1,False
1,0.0,0,0,0.0,26.0,26.0,1.0,4.0,5.0,0.153846,0.192308,1,0,True,0.0,-1,False


In [67]:
ss = ss.append(ss, ignore_index=True)
#ss = ss.append(ss, ignore_index=True)

In [70]:
scope = pd.Series(index=ss.index)
scope.loc[:]=True

/var/folders/lr/nrx49w_54md3dfrcjb0y_b580000gn/T/ipykernel_5806/3789991195.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scope = pd.Series(index=ss.index)


In [ ]:
scope.loc[1] = False
scope.loc[0] = False
scope.loc[6] = False

In [72]:
n_tasks = get_n_tasks(ss)

eligible_scope = scope
ss["request"] = eligible_scope.apply(lambda x: [True, False] if x else [False])

request_sates = ss["request"].values
request_sates

array([list([True, False]), list([True, False])], dtype=object)

In [73]:
request_sates = request_sates.reshape(
    (int(request_sates.shape[0] / n_tasks), n_tasks )
)
request_sates

array([[list([True, False]), list([True, False])]], dtype=object)

In [74]:
def f(x):
    return list(itertools.product(*x))

In [75]:
request_sates = np.array(list(map(f, request_sates)))
request_sates

array([[[ True,  True],
        [ True, False],
        [False,  True],
        [False, False]]])

In [76]:
max_combinations = 2**n_tasks

In [77]:
combinations_df = pd.DataFrame(index=range(int(ss.shape[0]/n_tasks)), columns=range(max_combinations))

In [78]:
combinations_df

,0,1,2,3
0,NaN,NaN,NaN,NaN


In [62]:
for i in range(len(request_sates)):
    request_sate = request_sates[i]
    combinations_df.iloc[i, :len(request_sate)] = list(request_sate)

In [63]:
combinations_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"[True, True, True, True]","[True, True, True, False]","[True, True, False, True]","[True, True, False, False]","[True, False, True, True]","[True, False, True, False]","[True, False, False, True]","[True, False, False, False]","[False, True, True, True]","[False, True, True, False]","[False, True, False, True]","[False, True, False, False]","[False, False, True, True]","[False, False, True, False]","[False, False, False, True]","[False, False, False, False]"
1,"[True, True, True, True]","[True, True, True, False]","[True, True, False, True]","[True, True, False, False]","[True, False, True, True]","[True, False, True, False]","[True, False, False, True]","[True, False, False, False]","[False, True, True, True]","[False, True, True, False]","[False, True, False, True]","[False, True, False, False]","[False, False, True, True]","[False, False, True, False]","[False, False, False, True]","[False, False, False, False]"


In [272]:
for col, combinations_series in combinations_df.iteritems():
    ss_scope = combinations_series.notna()
    if not ss_scope.any():
        break
    task_scope = ss_scope.repeat(n_tasks)
    request_values = list(itertools.chain.from_iterable(combinations_series.dropna()))

[False, False, True, True, True, True, False, True, True, True, True, True, True, True, True, True]
[False, False, True, False, True, True, False, False, True, True, True, False, True, True, True, False]
[False, False, False, True, True, False, False, True, True, True, False, True, True, True, False, True]
[False, False, False, False, True, False, False, False, True, True, False, False, True, True, False, False]
[False, True, False, True, True, False, True, True, True, False, True, True]
[False, True, False, False, True, False, True, False, True, False, True, False]
[False, False, False, True, True, False, False, True, True, False, False, True]
[False, False, False, False, True, False, False, False, True, False, False, False]
[False, True, True, True, False, True, True, True]
[False, True, True, False, False, True, True, False]
[False, True, False, True, False, True, False, True]
[False, True, False, False, False, True, False, False]
[False, False, True, True, False, False, True, True]

In [82]:
t = np.array([False, False, False, True, False, False, False, True])
np.insert(t, 0, 1)

array([ True, False, False, False,  True, False, False, False,  True])

In [734]:
%%time

ts = TaskSet.TaskSet(
    [
        Task.Task(O=0, T=6, D=6, X=1, C=(2, 2)),
        Task.Task(O=0, T=3, D=3, X=1, C=(1, 1)),
        Task.Task(O=0, T=5, D=5, X=1, C=(1, 1)),
        Task.Task(O=0, T=15, D=15, X=1, C=(1, 2)),
    ]
)

ss = get_initial_state(ts.get_df())

visited = set()
ss_hashes = get_hashes(ss)
visited_test = set()
ss_hashes_test = get_hashes_test(ss)
i = 0
n_tasks = get_n_tasks(ss)
while ss.shape[0] > 0:
    visited.update(ss_hashes)
    visited_test.update(ss_hashes_test)

    ss = get_neighbours(ss)

    if fail(ss):
        print("fail")
        break

    ss_hashes = ss["hash"].iloc[::n_tasks]

    ss_visited = ss_hashes.isin(visited)
    ss_visited.name = "visited"
    
    ss = ss.drop("visited", axis=1, errors="ignore")
    ss = ss.join(ss_visited)
    ss["visited"] = ss["visited"].fillna(method='ffill')
    ss = ss.loc[~ss["visited"]]
    

    print(i, ss.shape[0] / len(ts), ss_visited.sum(), len(visited))
    i += 1

0 15.0 1 1 1
1 91.0 0 16 1
2 314.0 0 107 1
3 632.0 232 421 1
4 983.0 0 1053 1
5 903.0 636 2036 1
6 737.0 632 2939 1
7 666.0 200 3676 1
8 652.0 75 4342 1
9 626.0 75 4994 1
10 592.0 102 5620 1
11 529.0 62 6212 1
12 524.0 106 6741 1
13 491.0 56 7265 1
14 482.0 24 7756 1
15 526.0 485 8238 1
16 298.0 559 8764 1
17 271.0 131 9062 1
18 243.0 206 9333 1
19 154.0 219 9576 1
20 115.0 182 9730 1
21 86.0 186 9845 1
22 61.0 102 9931 1
23 42.0 88 9992 1
24 56.0 94 10034 1
25 65.0 92 10090 1
26 48.0 84 10155 1
27 41.0 57 10203 1
28 29.0 22 10244 1
29 16.0 17 10273 1
30 0.0 92 10289 1
CPU times: user 1min 43s, sys: 410 ms, total: 1min 44s
Wall time: 1min 43s


In [493]:
ts = TaskSet.TaskSet(
    [
        Task.Task(O=0, T=3, D=3, X=1, C=(2, 2)),
        Task.Task(O=0, T=3, D=3, X=0, C=(1, 1)),
    ]
)

In [495]:
ss = get_initial_state(ts.get_df())

In [497]:
i = 0
test_set = set()
test_set_2 = set()

for nat in range(-1,int(ss["T"].iloc[1]+1)):
    for done in [False, True]:
        for rct in range(int(ss[1].iloc[1]+1)):
            for crit in range(2):

                for nat2 in range(-1,int(ss["T"].iloc[0]+1)):
                    for done2 in [False, True]:
                        for rct2 in range(int(ss[1].iloc[0]+1)):
                            for crit2 in range(2):

                                ss.loc[1, "nat"] = nat
                                ss.loc[1, "rct"] = rct
                                ss.loc[1, "crit"] = crit
                                ss.loc[1, "done"] = done

                                ss.loc[0, "nat"] = nat2
                                ss.loc[0, "rct"] = rct2
                                ss.loc[0, "crit"] = crit2
                                ss.loc[0, "done"] = done2

                                ss_hash = get_hashes_idle(ss)[0]
                                test_set.add(ss_hash)
                                test_set_2.add(tuple(ss["hash_value"]))
                                i+= 1
                                print(i, len(test_set_2), len(test_set), ss_hash)

1 1 1 0.0
2 2 2 1.0
3 3 3 2.0
4 4 4 3.0
5 5 5 4.0
6 6 6 5.0
7 7 7 6.0
8 8 8 7.0
9 9 9 8.0
10 10 10 9.0
11 11 11 10.0
12 12 12 11.0
13 13 13 12.0
14 14 14 13.0
15 15 15 14.0
16 16 16 15.0
17 17 17 16.0
18 18 18 17.0
19 18 18 6.0
20 18 18 7.0
21 18 18 8.0
22 18 18 9.0
23 18 18 10.0
24 18 18 11.0
25 19 19 24.0
26 20 20 25.0
27 21 21 26.0
28 22 22 27.0
29 23 23 28.0
30 24 24 29.0
31 24 24 6.0
32 24 24 7.0
33 24 24 8.0
34 24 24 9.0
35 24 24 10.0
36 24 24 11.0
37 25 25 36.0
38 26 26 37.0
39 27 27 38.0
40 28 28 39.0
41 29 29 40.0
42 30 30 41.0
43 30 30 6.0
44 30 30 7.0
45 30 30 8.0
46 30 30 9.0
47 30 30 10.0
48 30 30 11.0
49 31 31 48.0
50 32 32 49.0
51 33 33 50.0
52 34 34 51.0
53 35 35 52.0
54 36 36 53.0
55 36 36 6.0
56 36 36 7.0
57 36 36 8.0
58 36 36 9.0
59 36 36 10.0
60 36 36 11.0
61 37 37 60.0
62 38 38 61.0
63 39 39 62.0
64 40 40 63.0
65 41 41 64.0
66 42 42 65.0
67 43 43 66.0
68 44 44 67.0
69 45 45 68.0
70 46 46 69.0
71 47 47 70.0
72 48 48 71.0
73 49 49 72.0
74 50 50 73.0
75 51 51 74.0
76 

In [505]:
ss

,nat,rct,crit,O,T,D,X,0,1,U0,U1,t_id,ss_id,done,wl,request,hash_value,hash_factor,sched_id,hash
0,0.0,0.0,0,0.0,25.0,25.0,0.0,2.0,2.0,0.080000,0.080000,0,15,True,0.0,False,18.0,324.0,-1.0,7.434552e+09
1,0.0,0.0,0,0.0,26.0,26.0,1.0,4.0,5.0,0.153846,0.192308,1,15,True,0.0,False,36.0,672.0,-1.0,7.434552e+09
2,0.0,0.0,0,0.0,27.0,27.0,1.0,5.0,6.0,0.185185,0.222222,2,15,True,0.0,False,42.0,812.0,-1.0,7.434552e+09
3,0.0,0.0,0,0.0,27.0,27.0,1.0,5.0,6.0,0.185185,0.222222,3,15,True,0.0,False,42.0,812.0,-1.0,7.434552e+09
4,25.0,2.0,0,0.0,25.0,25.0,0.0,2.0,2.0,0.080000,0.080000,0,16,False,23.0,True,316.0,324.0,-1.0,1.405484e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.0,0.0,0,0.0,27.0,27.0,1.0,5.0,6.0,0.185185,0.222222,3,29,True,0.0,False,42.0,812.0,-1.0,7.598283e+09
60,0.0,0.0,0,0.0,25.0,25.0,0.0,2.0,2.0,0.080000,0.080000,0,30,True,0.0,False,18.0,324.0,-1.0,1.403845e+11
61,0.0,0.0,0,0.0,26.0,26.0,1.0,4.0,5.0,0.153846,0.192308,1,30,True,0.0,False,36.0,672.0,-1.0,1.403845e+11
62,0.0,0.0,0,0.0,27.0,27.0,1.0,5.0,6.0,0.185185,0.222222,2,30,True,0.0,False,42.0,812.0,-1.0,1.403845e+11


In [205]:
def get_task_hash_idle(ss):
    value = ss["crit"]
    factor = 2

    value += ss["rct"] * factor
    factor *= ss[1] + 1

    value += (ss["done"]).astype(int) * factor
    factor *= 2

    value += ((ss["nat"] + 1)*(~ss["done"]).astype(int)) * factor
    factor *= ss["T"] + 2

    ss["hash_value"] = value
    ss["hash_factor"] = factor

    return ss


def combine_task_hashes_idle(ss):
    combine_hash_factor = ss["hash_factor"]
    combine_hash_factor = combine_hash_factor.cumprod()
    nat_order_factor = ss["hash_factor"].iloc[-1]
    combine_hash_factor = combine_hash_factor.shift().fillna(1)
    ss_hash = (ss["hash_value"] * combine_hash_factor).sum()

    ss = ss.sort_values("nat")    
    
    nat_order_value = lexicographical_index_l(ss.index.get_level_values(1))
    ss_hash += nat_order_value*nat_order_factor
    return ss_hash


def get_hashes_idle(ss):
    ss = get_task_hash_idle(ss)
    ss_hashes = (
        ss.set_index(["ss_id", "t_id"])[["hash_value", "hash_factor", "nat"]]
        .groupby(level="ss_id")
        .apply(combine_task_hashes_idle)
    )

    ss_hashes.name = "hash_idle"

    return ss_hashes

In [9]:
def set_hashes_idle(ss):
    hashes = get_hashes_idle(ss)
    ss = ss.drop("hash_idle", axis=1, errors="ignore")
    ss = ss.merge(hashes, left_on="ss_id", right_index=True)
    return ss

In [518]:
ss = set_hashes_idle(ss)

In [342]:
class MaxSet:
    def __init__(self):
        self.set = dict()
        self.size = 0

    def add(self, ss, h):
        return self.addIdle(ss.reset_index(drop=True), h)

    def addIdle(self, new, h):
        try:
            currents = self.set[h]
        except KeyError:
            self.set[h] = [new.copy()]
            self.size += 1
            return True
        else:
            toReplace = []
            for i in range(len(currents)):
                
                current = currents[i]
                
                CurrentSimuleNew = simulate(current, new)

                if CurrentSimuleNew:
                    return False
                
                newSimuleCurrent = simulate(new, current)

                if newSimuleCurrent:
                    toReplace.append(i)
                    
            for i in toReplace[::-1]:
                del self.set[h][i]
                self.size -= 1
                
            self.set[h].append(new.copy())
            self.size += 1
            return True

    def __set__(self):
        return set(self.set.values())

    def __repr__(self):
        return set(self)

    def __iter__(self):
        return self.set.values().__iter__()

    def __len__(self):
        return len(self.set)

visited = MaxSet()

In [343]:
n_tasks

4

In [344]:
#%%time

ts = TaskSet.TaskSet(
    [
        Task.Task(O=0, T=6, D=6, X=1, C=(2, 2)),
        Task.Task(O=0, T=3, D=3, X=1, C=(1, 1)),
        Task.Task(O=0, T=5, D=5, X=1, C=(1, 1)),
        Task.Task(O=0, T=15, D=15, X=1, C=(1, 2)),
    ]
)

print(ts.getAverageUtilisation())
print(ts.getUtilisationOfLevel(1))

ss = get_initial_state(ts.get_df())
visited = MaxSet()

visited_all = set()

i = 0
n_tasks = get_n_tasks(ss)
ss = set_hashes_idle(ss)

visited.add(ss, ss["hash_idle"].iloc[0])


ss_hashes = get_hashes(ss)
visited_all.update(ss_hashes)

while ss.shape[0] > 0:

    ss = get_neighbours(ss)

    if fail(ss):
        print("fail")
        break
    
    ss = set_hashes_idle(ss)
    news_ss_scope = grouped.apply(lambda x: visited.add(x["nat"], x["hash_idle"].iloc[0]))
    news_ss_scope = news_ss_scope.repeat(n_tasks).values
    
    ss = ss.loc[news_ss_scope]
    ss_hashes = get_hashes(ss)
    visited_all.update(ss_hashes)
    

    print(i, ss.shape[0] / len(ts), (~news_ss_scope).sum()/ len(ts), len(visited), len(visited_all))
    i += 1

0.9666666666666667
1.0


KeyError: "None of [Index(['crit', 'done', 'rct'], dtype='object')] are in the [index]"

In [336]:
class MaxSet:
    def __init__(self):
        self.df = pd.DataFrame(columns=["key"])
        self.size = 0

    def add(self, nat, h):
        nat = nat.copy()
        nat = nat.reset_index(drop=True)
        return self.addIdle(nat, h)

    def addIdle(self, nat, h):
        key_scope = self.df["key"] == h
        if not key_scope.any():
            nat["key"] = h
            self.df = self.df.append(nat, ignore_index=True)
            return True
        simulated_scope = (self.df.loc[key_scope, range(nat.shape[0])] <= nat).all(axis=1)
        if simulated_scope.any():
            return False
        else:
            simulated_scope = (self.df.loc[key_scope, range(nat.shape[0])] >= nat).all(axis=1)
            simulated_rows = simulated_scope.loc[simulated_scope].index
            self.df = self.df.drop(simulated_rows)
            nat["key"] = h
            self.df = self.df.append(nat, ignore_index=True)
            
            
            return True
        

    def __repr__(self):
        return set(self.df)

    def __iter__(self):
        return self.df.__iter__()

    def __len__(self):
        return len(self.df)

visited = MaxSet()

In [337]:
%%time

ts = TaskSet.TaskSet(
    [
        Task.Task(O=0, T=25, D=25, X=1, C=(2, 21)),
        Task.Task(O=0, T=26, D=26, X=0, C=(2, 2)),
        Task.Task(O=0, T=27, D=27, X=1, C=(1, 2)),
        Task.Task(O=0, T=34, D=34, X=1, C=(1, 3)),
    ]
)

print(ts.getAverageUtilisation())
print(ts.getUtilisationOfLevel(1))

ss = get_initial_state(ts.get_df())
visited = MaxSet()

visited_all = set()

i = 0
n_tasks = get_n_tasks(ss)
ss = set_hashes_idle(ss)

visited.add(ss["nat"], ss["hash_idle"].iloc[0])


ss_hashes = get_hashes(ss)
visited_all.update(ss_hashes)

while ss.shape[0] > 0:

    ss = get_neighbours(ss)

    if fail(ss):
        print("fail")
        break
    
    ss = set_hashes_idle(ss)
    grouped = ss[["nat", "hash_idle", "ss_id"]].groupby("ss_id")
    news_ss_scope = grouped.apply(lambda x: visited.add(x["nat"], x["hash_idle"].iloc[0]))
    news_ss_scope = news_ss_scope.repeat(n_tasks).values
    
    ss = ss.loc[news_ss_scope]
    ss_hashes = get_hashes(ss)
    visited_all.update(ss_hashes)
    

    print(i, ss.shape[0] / len(ts), (~news_ss_scope).sum()/ len(ts), len(visited), len(visited_all))
    i += 1

0.6128406234288587
1.002309368191721
0 15.0 1.0 16 16
1 116.0 4.0 132 132
2 317.0 125.0 412 449
3 508.0 348.0 778 957
4 589.0 491.0 1081 1546
5 583.0 597.0 1342 2129


KeyboardInterrupt: 

In [943]:
ss

,nat,rct,crit,O,T,D,X,0,1,U0,...,t_id,ss_id,done,wl,request,hash_value,hash_factor,sched_id,hash,hash_idle
0,23.0,19.0,1,0.0,25.0,25.0,1.0,2.0,21.0,0.080000,...,0,16,False,4.0,False,2151.0,2376.0,0,1.488873e+11,1.488873e+11
1,0.0,0.0,1,0.0,26.0,26.0,0.0,2.0,2.0,0.076923,...,1,16,True,0.0,False,19.0,336.0,0,1.488873e+11,1.488873e+11
2,25.0,2.0,1,0.0,27.0,27.0,1.0,1.0,2.0,0.037037,...,2,16,False,23.0,False,317.0,348.0,0,1.488873e+11,1.488873e+11
3,32.0,3.0,1,0.0,34.0,34.0,1.0,1.0,3.0,0.029412,...,3,16,False,29.0,False,535.0,576.0,0,1.488873e+11,1.488873e+11
4,23.0,19.0,1,0.0,25.0,25.0,1.0,2.0,21.0,0.080000,...,0,18,False,4.0,False,2151.0,2376.0,0,7.198643e+09,2.753480e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,33.0,1.0,0,0.0,34.0,34.0,1.0,1.0,3.0,0.029412,...,3,556,False,30.0,False,546.0,576.0,0,1.517054e+11,1.516958e+11
1368,23.0,0.0,0,0.0,25.0,25.0,1.0,2.0,21.0,0.080000,...,0,557,True,23.0,False,2156.0,2376.0,0,1.519601e+11,1.519601e+11
1369,0.0,0.0,0,0.0,26.0,26.0,0.0,2.0,2.0,0.076923,...,1,557,True,0.0,False,18.0,336.0,0,1.519601e+11,1.519601e+11
1370,27.0,1.0,0,0.0,27.0,27.0,1.0,1.0,2.0,0.037037,...,2,557,False,25.0,True,338.0,348.0,0,1.519601e+11,1.519601e+11


In [948]:
dfv = visited.df.melt(id_vars=['key'], var_name="t_id")

In [949]:
dfv

,key,t_id,value
0,2.227372e+09,0,0.0
1,1.564060e+11,0,25.0
2,2.493187e+09,0,25.0
3,1.561409e+11,0,25.0
4,2.228139e+09,0,25.0
...,...,...,...
1043,2.483604e+09,3,0.0
1044,1.563956e+11,3,34.0
1045,1.519608e+11,3,33.0
1046,1.516958e+11,3,33.0


In [923]:
visited.df

,key,0,1,2,3
0,1305990.0,0.0,0.0,0.0,0.0
1,41913112.0,6.0,3.0,5.0,15.0
2,1485592.0,6.0,3.0,5.0,0.0
3,41736472.0,6.0,3.0,0.0,15.0
4,1308952.0,6.0,3.0,0.0,0.0
...,...,...,...,...,...
976,1524999.0,2.0,1.0,1.0,1.0
977,8406247.0,1.0,1.0,1.0,2.0
978,10986727.0,1.0,1.0,1.0,3.0
979,1609447.0,1.0,1.0,2.0,1.0


In [991]:
df_merge = ss.merge(dfv, left_on=["hash_idle", "t_id"], right_on=["key", "t_id"], how="left")
simul = (df_merge["value"] <= df_merge["nat"]).values
simul = simul.reshape((int(simul.shape[0]/4), 4))
simul = np.logical_and.reduce(simul, axis=1)
simul_ss_id = df_merge.loc[::4].loc[simul, "ss_id"]
simul_ss_id.name="ss_id_visited"
ss = ss.merge(simul_ss_id, left_on="ss_id", right_on="ss_id_visited", how="left")
ss = ss.loc[ss["ss_id_visited"].isna()]

In [992]:
simul = simul.reshape((int(simul.shape[0]/4), 4))

In [993]:
simul = np.logical_and.reduce(simul, axis=1)

In [994]:
simul_ss_id = df_merge.loc[::4].loc[simul, "ss_id"]

In [1002]:
simul_ss_id.name="ss_id_visited"

In [1004]:
ss = ss.merge(simul_ss_id, left_on="ss_id", right_on="ss_id_visited", how="left")

In [1006]:
ss = ss.loc[ss["ss_id_visited"].isna()]

,nat,rct,crit,O,T,D,X,0,1,U0,...,ss_id,done,wl,request,hash_value,hash_factor,sched_id,hash,hash_idle,ss_id_visited


In [1203]:
%%time

ts = TaskSet.TaskSet(
    [
        Task.Task(O=0, T=25, D=25, X=1, C=(2, 21)),
        Task.Task(O=0, T=26, D=26, X=0, C=(2, 2)),
        Task.Task(O=0, T=27, D=27, X=1, C=(1, 2)),
        Task.Task(O=0, T=34, D=34, X=1, C=(1, 3)),
    ]
)

print(ts.getAverageUtilisation())
print(ts.getUtilisationOfLevel(1))

ss = get_initial_state(ts.get_df())

visited_all = set()
visited = pd.DataFrame()


i = 0
n_tasks = get_n_tasks(ss)
ss = set_hashes_idle(ss)

ss_hashes = get_hashes(ss)
visited_all.update(ss_hashes)

0.6128406234288587
1.002309368191721
CPU times: user 21.1 ms, sys: 2.41 ms, total: 23.5 ms
Wall time: 21.6 ms


In [1204]:
n_tasks = 4

In [1205]:
n_tasks = 4

visited = ss[["hash_idle", "nat", "t_id"]].rename(columns = {"nat":"nat_idle"})
visited["item_id"] = 0

In [1206]:
visited

,hash_idle,nat_idle,t_id,item_id
0,2.227372e+09,0.0,0,0
1,2.227372e+09,0.0,1,0
2,2.227372e+09,0.0,2,0
3,2.227372e+09,0.0,3,0


In [1207]:
ss

,nat,rct,crit,O,T,D,X,0,1,U0,U1,t_id,ss_id,done,wl,sched_id,request,hash_value,hash_factor,hash_idle
0,0.0,0,0,0.0,25.0,25.0,1.0,2.0,21.0,0.080000,0.840000,0,0,True,0.0,-1,False,132.0,2376.0,2.227372e+09
1,0.0,0,0,0.0,26.0,26.0,0.0,2.0,2.0,0.076923,0.076923,1,0,True,0.0,-1,False,18.0,336.0,2.227372e+09
2,0.0,0,0,0.0,27.0,27.0,1.0,1.0,2.0,0.037037,0.074074,2,0,True,0.0,-1,False,18.0,348.0,2.227372e+09
3,0.0,0,0,0.0,34.0,34.0,1.0,1.0,3.0,0.029412,0.088235,3,0,True,0.0,-1,False,24.0,576.0,2.227372e+09


In [321]:
def remove_simulated_list(ss, df_merge, simulated, id_col):
    simulated_ss_id = df_merge.loc[::n_tasks].loc[simulated, id_col]
    simulated_ss_id= simulated_ss_id.drop_duplicates()
    simulated_ss_id.name="ss_id_simulated"
    
    ss = ss.merge(simulated_ss_id, left_on="ss_id", right_on="ss_id_simulated", how="left")
    ss = ss.loc[ss["ss_id_simulated"].isna()]
    ss = ss.drop("ss_id_simulated", axis=1)
    
    return ss

In [133]:
def append_visited_states(visited, other, n_tasks):
    ss_id_start = int(visited["ss_id"].max()+1)
    other["ss_id"] = np.array(range(ss_id_start, int(other.shape[0]/n_tasks)+ss_id_start)).repeat(n_tasks)
    visited = visited.append(other, ignore_index=True)
    return visited

In [317]:
def remove_self_simulated(ss, n_tasks):
    df_merge = ss.merge(ss, on=["hash_idle", "t_id"], how="left", suffixes=("","_other"))
    df_merge = df_merge.sort_values(["hash_idle", "ss_id", "ss_id_other"])
    
    simulated = (df_merge["nat_other"] <= df_merge["nat"])
    simulated &= (df_merge["ss_id"] != df_merge["ss_id_other"])
    simulated = simulated.values
        
    simulated = simulated.reshape((int(simulated.shape[0]/n_tasks), n_tasks))
    simulated = np.logical_and.reduce(simulated, axis=1)
    
    ss = remove_simulated_list(ss, df_merge, simulated, id_col="ss_id")
    
    return ss

In [360]:
def remove_visited_simulation(ss, visited, n_tasks):
    idx_l = ["hash_idle", "t_id"]
    df_merge = ss.set_index(idx_l).join(visited.set_index(idx_l), how="left", rsuffix="_other")
    df_merge = df_merge.reset_index()
    df_merge = df_merge.sort_values(["hash_idle", "ss_id", "ss_id_other"])
    
    simulated_full = (df_merge["nat_other"] <= df_merge["nat"])
    simulated = simulated_full.values.reshape((int(simulated_full.shape[0]/n_tasks), n_tasks))
    simulated = np.logical_and.reduce(simulated, axis=1)
    
    ss = remove_simulated_list(ss, df_merge, simulated, id_col="ss_id")
    
    never_seen_scope = df_merge["nat_other"].isna()
    never_seen_states = df_merge[never_seen_scope][["hash_idle", "nat", "t_id"]]
    
    df_merge = df_merge[~never_seen_scope]
    
    simulator = ((df_merge["nat"] <= df_merge["nat_other"])).values
    simulator = simulator.reshape((int(simulator.shape[0]/n_tasks), n_tasks))
    simulator = np.logical_and.reduce(simulator, axis=1)
    
    different = ((df_merge["nat"] != df_merge["nat_other"])).values
    different = different.reshape((int(different.shape[0]/n_tasks), n_tasks))
    different = np.logical_or.reduce(different, axis=1)
    
    simulator &= different
    
    visited = remove_simulated_list(visited, df_merge, simulator, id_col="ss_id_other")

    simulator_states = df_merge.loc[simulator.repeat(n_tasks)][["hash_idle", "nat", "t_id"]]
    
    visited = append_visited_states(visited, simulator_states, n_tasks)
    visited = append_visited_states(visited, never_seen_states, n_tasks)
    
    return ss, visited

In [11]:
import numpy as np

In [335]:
%%time

ts = TaskSet.TaskSet(
    [
        Task.Task(O=0, T=25, D=25, X=1, C=(2, 21)),
        Task.Task(O=0, T=26, D=26, X=0, C=(2, 2)),
        Task.Task(O=0, T=27, D=27, X=1, C=(1, 2)),
        Task.Task(O=0, T=34, D=34, X=1, C=(1, 3)),
    ]
)

ss = get_initial_state(ts.get_df())
ss = set_hashes_idle(ss)

print(ts.getAverageUtilisation())
print(ts.getUtilisationOfLevel(1))

n_tasks = len(ts)

visited = ss[["hash_idle", "nat", "t_id"]]
visited["ss_id"] = 0

i = 0

while ss.shape[0] > 0:

    ss = get_neighbours(ss)
    ss = set_hashes_idle(ss)

    if fail(ss):
        print("fail")
        break
       
    
    ss = remove_self_simulated(ss, n_tasks)
    ss, visited= remove_visited_simulation(ss, visited, n_tasks)

    print(i, len(ss)/n_tasks, len(visited)/n_tasks, visited.groupby("hash_idle").count().max().max())
    i += 1

0.6128406234288587
1.002309368191721
0 15.0 16.0 4


<timed exec>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1 116.0 132.0 4
2 317.0 406.0 4
3 473.0 751.0 4
4 522.0 1027.0 8
5 518.0 1271.0 8
6 561.0 1535.0 8
7 646.0 1839.0 8
8 745.0 2199.0 8
9 852.0 2607.0 8
10 963.0 3059.0 8
11 1082.0 3555.0 8
12 1205.0 4094.0 8
13 1337.0 4683.0 8
14 1476.0 5324.0 8
15 1622.0 6019.0 8
16 1775.0 6770.0 8
17 1935.0 7579.0 8
18 2102.0 8448.0 8
19 2275.0 9378.0 8
20 2454.0 10370.0 8
21 2111.0 10897.0 8
22 1595.0 11308.0 8
23 971.0 11630.0 8
24 640.0 11801.0 8
25 439.0 11928.0 8
26 382.0 12001.0 8
27 433.0 12159.0 8
28 334.0 12321.0 8
29 358.0 12501.0 8
30 369.0 12704.0 8
31 366.0 12924.0 8
32 354.0 13145.0 8
33 369.0 13372.0 8
34 439.0 13668.0 8
35 425.0 13990.0 8
36 450.0 14336.0 8
37 472.0 14704.0 8
38 490.0 15092.0 8
39 504.0 15498.0 8
40 513.0 15920.0 8
41 518.0 16356.0 8
42 534.0 16819.0 8
43 563.0 17294.0 8
44 594.0 17778.0 8
45 610.0 18274.0 8
46 530.0 18671.0 8
47 461.0 18760.0 8
48 474.0 18810.0 8
49 156.0 18816.0 8
50 177.0 18911.0 8
51 167.0 19007.0 8
52 166.0 19080.0 8
53 139.0 19147.0 8
54 188.0 192

In [506]:
%%time

ts = TaskSet.TaskSet(
    [
        Task.Task(O=0, T=25, D=25, X=1, C=(2, 21)),
        Task.Task(O=0, T=26, D=26, X=0, C=(2, 2)),
        Task.Task(O=0, T=27, D=27, X=1, C=(1, 2)),
        Task.Task(O=0, T=34, D=34, X=1, C=(1, 3)),
    ]
)

ss = get_initial_state(ts.get_df())
ss = set_hashes_idle(ss)

print(ts.getAverageUtilisation())
print(ts.getUtilisationOfLevel(1))

n_tasks = len(ts)

visited = ss[["hash_idle", "nat", "t_id"]]
visited["ss_id"] = 0

i = 0

while ss.shape[0] > 0:

    ss = get_neighbours(ss)
    ss = set_hashes_idle(ss)

    if fail(ss):
        print("fail")
        break
       
    
    ss = remove_self_simulated(ss, n_tasks)
    ss, visited= remove_visited_simulation(ss, visited, n_tasks)

    print(i, len(ss)/n_tasks, len(visited)/n_tasks, visited.groupby("hash_idle").count().max().max())
    i += 1

0.6128406234288587
1.002309368191721
0 15.0 16.0 4


<timed exec>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/simonpicard/Documents/mcae/sporadic.py:266: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  request_sates = np.array(list(map(f, request_sates)))


1 116.0 132.0 4
2 317.0 406.0 4
3 473.0 751.0 4
4 522.0 1027.0 8
5 518.0 1271.0 8
6 561.0 1535.0 8
7 646.0 1839.0 8
8 745.0 2199.0 8
9 852.0 2607.0 8
10 963.0 3059.0 8
11 1082.0 3555.0 8
12 1205.0 4094.0 8
13 1337.0 4683.0 8
14 1476.0 5324.0 8
15 1622.0 6019.0 8
16 1775.0 6770.0 8
17 1935.0 7579.0 8
18 2102.0 8448.0 8
19 2275.0 9378.0 8
20 2454.0 10370.0 8
21 2111.0 10897.0 8
22 1595.0 11308.0 8
23 971.0 11630.0 8
24 640.0 11801.0 8
25 439.0 11928.0 8
26 382.0 12001.0 8
27 433.0 12159.0 8
28 334.0 12321.0 8
29 358.0 12501.0 8
30 369.0 12704.0 8
31 366.0 12924.0 8
32 354.0 13145.0 8
33 369.0 13372.0 8
34 439.0 13668.0 8
35 425.0 13990.0 8
36 450.0 14336.0 8
37 472.0 14704.0 8
38 490.0 15092.0 8
39 504.0 15498.0 8
40 513.0 15920.0 8
41 518.0 16356.0 8
42 534.0 16819.0 8
43 563.0 17294.0 8
44 594.0 17778.0 8
45 610.0 18274.0 8
46 530.0 18671.0 8
47 461.0 18760.0 8
48 474.0 18810.0 8
49 156.0 18816.0 8
50 177.0 18911.0 8
51 167.0 19007.0 8
52 166.0 19080.0 8
53 139.0 19147.0 8
54 188.0 192

In [5]:
from math import factorial
from functools import reduce
from collections import Counter

def lexicographical_index(idx):
    rank = 0
    l = len(idx)
    freqs = [1]*l
    
    for n in range(l):
        fsum = sum([freqs[j] for j in range(idx[n])])
        freqs[idx[n]] -= 1;
        rank += (fsum * factorial(l-n-1))
    return rank

In [10]:
import multiprocessing

multiprocessing.cpu_count()

8

In [12]:
import pandas as pd

In [55]:
t = pd.DataFrame(range(10))

In [51]:
t["A"] = range(10)

In [35]:
scope = t["A"]%2 == 0

In [56]:
scope = t.index%2 == 0

In [57]:
tr = [[True, False]]*scope.sum()
t.loc[scope, 0] = tr

ValueError: Must have equal len keys and value when setting with an ndarray

In [58]:
t

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9
